In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os

import mysql.connector

In [ ]:
mysql_password = os.environ.get("MYSQL_PASSWORD")
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password
)
mycursor = mydb.cursor(dictionary=True)

mycursor.execute("USE webscraper;")

In [ ]:
manufacturer_bar_data = []
# finding the amount of each manufacturer from the database
mycursor.execute("SELECT manufacturer_id, COUNT(*) as count FROM dataset GROUP BY manufacturer_id ORDER BY count DESC")
results = mycursor.fetchall()
for row in results:
    manufacturer_bar_data.append(row)

# splitting the manufacturer_ids from their respective counts
manufacturer_ids = np.array([entry['manufacturer_id'] for entry in manufacturer_bar_data])
counts = np.array([entry['count'] for entry in manufacturer_bar_data])

# finding the table of manufacturer_id and manufacturer_name
mycursor.execute("SElECT manufacturer_id, manufacturer_name from manufacturers")
manufacturer_name_data = mycursor.fetchall()

# matching the id with the manufacturer's name from the database
manufacturer_labels = []
for i in range(len(manufacturer_ids)):
    for j in range(len(manufacturer_name_data)):
        if manufacturer_ids[i] == manufacturer_name_data[j]['manufacturer_id']:
            manufacturer_labels.append(manufacturer_name_data[j]['manufacturer_name'])

# plotting the bar chart
plt.figure(figsize=(15,7))
bars = plt.bar(manufacturer_labels, counts)
for bar, count in zip(bars, counts):
    plt.text(bar.get_x() + bar.get_width() / 2, count, str(count), ha='center', va='bottom')
plt.title("Distribution of manufacturers by volume")
plt.xlabel("Manufacturers"), plt.ylabel("Amount of listings"), plt.xticks(rotation=90)
plt.show()

In [ ]:
model_bar_data = []
# finding the top 25 models by amount from the database
mycursor.execute("SELECT model_id, COUNT(*) as count FROM dataset GROUP BY model_id ORDER BY count DESC LIMIT 25")
results = mycursor.fetchall()
for row in results:
    model_bar_data.append(row)

# splitting the model_id from their respective counts
model_ids = np.array([entry['model_id'] for entry in model_bar_data])
counts = np.array([entry['count'] for entry in model_bar_data])

# finding the table of model_id and model_name
mycursor.execute("SELECT model_id, model_name FROM models")
model_name_data = mycursor.fetchall()

# matching the model_id with the model's name from the database
labels = []
for i in range(len(model_ids)):
    for j in range(len(model_name_data)):
        if model_ids[i] == model_name_data[j]['model_id']:
            labels.append(model_name_data[j]['model_name'])

# plotting the bar chart
plt.figure(figsize=(15,7))
bars = plt.bar(labels, counts)
for bar, count in zip(bars, counts):
    plt.text(bar.get_x() + bar.get_width() / 2, count, str(count), ha='center', va='bottom')
plt.title("Top 25 models by volume")
plt.xlabel("Model name"), plt.ylabel("Amount of listings")
plt.xticks(rotation=90)
plt.show() 

In [ ]:
# distribution of fuel types for each manufacturer

petrol = []
diesel = []
hybrid = []
for manufacturer in manufacturer_ids:
    petrol_query = f"SELECT * FROM dataset WHERE fuel_type_id = 1 AND manufacturer_id = {manufacturer}"
    mycursor.execute(petrol_query)
    results = mycursor.fetchall()
    if results:
        petrol.append(results)
    else:
        print(manufacturer)
    
    diesel_query = f"SELECT * FROM dataset WHERE fuel_type_id = 2 AND manufacturer_id = {manufacturer}"
    mycursor.execute(diesel_query)
    results = mycursor.fetchall()
    if results:
        diesel.append(results)
    else:
        print(manufacturer)
    
    hybrid_query =f"SELECT * FROM dataset WHERE fuel_type_id IN (3, 4, 5, 6) AND manufacturer_id = {manufacturer}"
    mycursor.execute(hybrid_query)
    results = mycursor.fetchall()
    if results:
        hybrid.append(results)
    else:
        print(manufacturer)

petrol_counts = []
for i in range(len(petrol)):
    petrol_counts.append(len(petrol[i]))

print(len(petrol_counts))
# find for each fuel type which manufacturers arent present. Then discount from list of manufacturers then graph. then find how to combine graphs.




# plt.figure(figsize=(15,7))
# plt.bar(manufacturer_labels, petrol_counts)
# plt.show()